In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus']=False
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (8, 6)

In [7]:
df =pd.read_csv('../data/grade.csv')
df

,학년,과목,결과,중간,기말
0,1학년,국어,좋음,80,90
1,1학년,국어,나쁨,50,40
2,1학년,국어,나쁨,20,50
3,1학년,수학,좋음,83,95
4,1학년,수학,좋음,93,86
5,2학년,국어,나쁨,44,65
6,2학년,국어,좋음,95,98
7,2학년,수학,좋음,96,99
8,2학년,수학,나쁨,57,69


In [10]:
df.info()

# int인 값들을 사용

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학년      9 non-null      object
 1   과목      9 non-null      object
 2   결과      9 non-null      object
 3   중간      9 non-null      int64 
 4   기말      9 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 488.0+ bytes


### pivot_table과 groupby 모두 조건부 통계량을 기준으로 데이터를 집계한다는 점에서 완전히 동일
### 하지만 출력물 구조의 차이가 있으므로, 상황에 맞는 함수 선택이 필요

### 보통은 출력 결과 자체가 결과물인 경우에는 pivot_table
### 중간 산출물인 경우에는 groupby 

### <주요 입력>
### data : 데이터프레임
### index : 행에 들어갈 조건
### columns : 열에 들어갈 조건
### values : 집계대상 칼럼 목록
### aggfunc : 집계함수 - 디폴트는 mean

In [15]:
# 피벗 테이블은 데이터를 조건에 따른 변수들의 통계량을 요약한 테이블

# 평균이 디폴트값

gradeDF = df.pivot_table(index='학년')
gradeDF[['중간', '기말']] # 순서지정

,중간,기말
학년,,
1학년,65.2,72.20
2학년,73.0,82.75


In [11]:
# 혹은 groupby

df.groupby('학년').mean()

,중간,기말
학년,,
1학년,65.2,72.20
2학년,73.0,82.75


In [12]:
# aggfunc

gradeDF = df.pivot_table(index='학년', aggfunc='sum')
gradeDF

,기말,중간
학년,,
1학년,361,326
2학년,331,292


In [13]:
# 학년별 과목별 집계

gradeDF = df.pivot_table(index=['학년', '과목'])
gradeDF

기말    중간
학년  과목            
1학년 국어  60.0  50.0
    수학  90.5  88.0
2학년 국어  81.5  69.5
    수학  84.0  76.5

In [14]:
# 혹은 groupby

g=df.groupby(['학년','과목'])
g.mean()

중간    기말
학년  과목            
1학년 국어  50.0  60.0
    수학  88.0  90.5
2학년 국어  69.5  81.5
    수학  76.5  84.0

In [19]:
# values : 집계대상 칼럼 목록

gradeDF = df.pivot_table(index=['학년', '과목'], values='중간')
gradeDF

중간
학년  과목      
1학년 국어  50.0
    수학  88.0
2학년 국어  69.5
    수학  76.5

In [18]:
# columns : 열에 들어갈 조건

# values와 columns가 둘다 있는 것은?
# 학년별 과목별 결과별 중간 점수의 평균
# index별 - columns별 - values의 - aggfunc

gradeDF = df.pivot_table(index=['학년', '과목'], values='중간', columns='결과')
gradeDF

결과        나쁨    좋음
학년  과목            
1학년 국어  35.0  80.0
    수학   NaN  88.0
2학년 국어  44.0  95.0
    수학  57.0  96.0

In [23]:
gradeDF = df.pivot_table(index='학년', aggfunc=['sum', 'mean'])
gradeDF

sum        mean      
      기말   중간     기말    중간
학년                        
1학년  361  326  72.20  65.2
2학년  331  292  82.75  73.0

In [36]:
# 시리즈로 넘어감
# 1학년 중간점수 / 2학년 중간점수

def fn(v):
#     print('=========')
#     print(v)
#     print('=========')
    if v.mean() >= 70:
        return '합격'
    else:
        return '불합격'
#     return v.mean()

In [37]:
# aggfunc인자에 함수를 줘도 됨

gradeDF = df.pivot_table(index='학년', values='중간', aggfunc=fn)
gradeDF

,중간
학년,
1학년,불합격
2학년,합격


In [30]:
# 혹은 aggfunc인자에 람다를 사용

gradeDF = df.pivot_table(index = '학년', 
                         values = '중간', 
                         aggfunc = lambda v:'합격' if v.mean()>70 else '불합격')
gradeDF

,중간
학년,
1학년,불합격
2학년,합격
